In [1]:
code_loc = "/home/shi/WorkSpace/projects/scMultiNet_workspace/"
root = "/home/shi/WorkSpace/projects/scMultiNet_Data/"
save_root = root + "/Step_1_data/Dataset_scLung/" 
import sys
sys.path.append(code_loc ) 

In [2]:
# start with a vocab list
# this related with the gene2vec model
#----> pre-trained part 
vocab_loc = code_loc +"/support_data/vocab_gene2vec_16906.pkl"
target_label = 'Ground_Truth' # the label to be predicted

In [3]:

csv_path = root+"/Step_0_data/lung_cancer_scrna_treatment.csv"
data_path = root +  "/Step_0_data/2024_03_13_EMT_sc_lung_cancer_scrna_treatment.h5ad"


In [4]:
import pandas as pd

df = pd.read_csv(csv_path)

In [5]:
df

,Unnamed: 0,A10_1001000407,A10_1001000408,A10_1001000412,A10_B003518,A10_B003523,A10_B003527,A10_B003529,A10_B003588,A11_1001000407,...,O15_B003183,O1_B001546,O20_B003179,O8_B001544,P10_B001546,P12_B003179,P14_B003179,P17_B003183,P21_B003179,P8_B001546
0,A1BG,0,0,0,0,0,0,7,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,A1BG-AS1,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,A1CF,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,A2M,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,14.0,0.0,0.0
4,A2M-AS1,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11924,LOC101928989,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11925,LOC101928992,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11926,LOC101928994,0,0,0,0,0,0,0,135,0,...,39.0,0.0,28.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0
11927,LOC101928995,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
mask = df.isna().any(axis = 1)
df = df[~mask]

gene_id = df.iloc[:,0].values
sample_id = df.columns[1:].values
data_x = df.iloc[:,1:].values

import numpy as np
# confirm nan and inf in data_x
print(np.isnan(data_x).any(), np.isinf(data_x).any())
# set nan to 0
data_x = np.nan_to_num(data_x)
# set inf to 0
data_x = np.where(data_x == np.inf, 0, data_x)

# normalize the data to 0-100
# 找到数据的最小值和最大值
#min_val = np.min(data_x)
#max_val = np.max(data_x)
#print(min_val, max_val )
# 将数据归一化到 0 到 100 的范围
#normalized_data_x = 100 * (data_x - min_val) / (max_val - min_val)
#print(normalized_data_x.shape)
# create the adata matrix
import anndata
adata = anndata.AnnData(X = data_x.T, obs = pd.DataFrame(index = sample_id), var = pd.DataFrame(index = gene_id))
adata.obs["Ground_Truth"] = np.random.choice([0,1], adata.shape[0])
print(adata, adata.obs, adata.var, adata.X)

False False
AnnData object with n_obs × n_vars = 3754 × 11928
    obs: 'Ground_Truth'                 Ground_Truth
A10_1001000407             0
A10_1001000408             1
A10_1001000412             0
A10_B003518                1
A10_B003523                1
...                      ...
P12_B003179                1
P14_B003179                1
P17_B003183                0
P21_B003179                1
P8_B001546                 1

[3754 rows x 1 columns] Empty DataFrame
Columns: []
Index: [A1BG, A1BG-AS1, A1CF, A2M, A2M-AS1, A2ML1, A2MP1, A3GALT2, A4GALT, A4GNT, AA06, AAAS, AACS, AACSP1, AADAC, AADACL2, AADACL2-AS1, AADACL3, AADACL4, AADACP1, AADAT, AAED1, AAGAB, AAK1, AAMDC, AAMP, AANAT, AAR2, AARD, AARS, AARS2, AARSD1, AASDH, AASDHPPT, AASS, AATBC, AATF, AATK, AATK-AS1, ABALON, ABAT, ABCA1, ABCA10, ABCA11P, ABCA12, ABCA13, ABCA17P, ABCA2, ABCA3, ABCA4, ABCA5, ABCA6, ABCA7, ABCA8, ABCA9, ABCA9-AS1, ABCB1, ABCB10, ABCB11, ABCB4, ABCB5, ABCB6, ABCB7, ABCB8, ABCB9, ABCC1, ABCC10, ABCC11,

In [7]:
adata.obs

,Ground_Truth
A10_1001000407,0
A10_1001000408,1
A10_1001000412,0
A10_B003518,1
A10_B003523,1
...,...
P12_B003179,1
P14_B003179,1
P17_B003183,0
P21_B003179,1


In [8]:
#adata.write(data_path)

In [9]:
# check the data
import scanpy as sc
import pandas as pd
import numpy as np
adata = sc.read(data_path)

print(adata)
print(adata.obs.index)
print(adata.var.index)
print(f" negative values: {np.sum(adata.X<0)}")

AnnData object with n_obs × n_vars = 3754 × 11929
    obs: 'Ground_Truth'
Index(['A10_1001000407', 'A10_1001000408', 'A10_1001000412', 'A10_B003518',
       'A10_B003523', 'A10_B003527', 'A10_B003529', 'A10_B003588',
       'A11_1001000407', 'A11_1001000408',
       ...
       'O15_B003183', 'O1_B001546', 'O20_B003179', 'O8_B001544', 'P10_B001546',
       'P12_B003179', 'P14_B003179', 'P17_B003183', 'P21_B003179',
       'P8_B001546'],
      dtype='object', length=3754)
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1',
       'A3GALT2', 'A4GALT', 'A4GNT',
       ...
       'LOC101928973', 'LOC101928977', 'LOC101928978', 'LOC101928979',
       'LOC101928988', 'LOC101928989', 'LOC101928992', 'LOC101928994',
       'LOC101928995', 'LOC101929019'],
      dtype='object', length=11929)
 negative values: 0


In [10]:
# adata.X 有多少nan
import numpy as np
from scipy.sparse import issparse

# 假设 adata.X 是你的稀疏矩阵
# 首先，检查它是否为稀疏矩阵
if issparse(adata.X):
    # 将稀疏矩阵转换为密集格式，然后计算 NaN 值的数量
    nan_count = np.isnan(adata.X.toarray()).sum()
else:
    # 如果 adata.X 不是稀疏矩阵，直接计算 NaN 值的数量
    nan_count = np.isnan(adata.X).sum()

print(f"NaN values in adata.X: {nan_count}")


NaN values in adata.X: 0


In [18]:

from scLLM.Dataset.paras import Dataset_para
# define pre-processing by follow original implementation of scBERT

from scLLM.Dataset.paras import Dataset_para
# define pre-processing by follow original implementation of scBERT 
dataset_para_cls = Dataset_para(
                            var_idx=None,
                            obs_idx="Ground_Truth",
                            vocab_loc=code_loc + "/Experiment/support_data/vocab_16k.json",
                            filter_gene_by_counts=False,
                            filter_cell_by_counts=0,
                            log1p=True,
                            log1p_base=2,

                            #
                            tokenize_name="scBERT",
                            cls_nb=5,
                            data_layer_name="X_log1p",
                            label_key = target_label,#"Ground_truth",#"Ground_truth",

                            test_size=None,#0.2, #use all data to inference
                            binarize=None, # not binarize use original label

                            )

dataset_para_reg = Dataset_para(
        vocab_loc=code_loc +"/Experiment/support_data/vocab_16k.json",
        var_idx = None,#"genes.gene_short_name",
        obs_idx="pseudotimes",
        filter_gene_by_counts=False,
        filter_cell_by_counts=200,
        log1p=True,
        log1p_base=2,

        tokenize_name="scBERT",
        cls_nb=1,
        data_layer_name="X_log1p",

        auto_map_str_labels=False,
        label_key = target_label,#"pseudotimes",

        test_size=None,#0.2, #use all data to inference
        binarize=None, # not binarize use original label for regression
    )

dataset_para = dataset_para_cls#dataset_para_cls
print(dataset_para)

Dataset_para(var_idx=None, obs_idx='Ground_Truth', vocab_loc='/home/shi/WorkSpace/projects/scMultiNet_workspace//Experiment/support_data/vocab_16k.json', gene_vocab=None, use_key='X', filter_gene_by_counts=False, filter_cell_by_counts=0, normalize_total=10000.0, result_normed_key='X_normed', log1p=True, result_log1p_key='X_log1p', log1p_base=2, subset_hvg=False, hvg_use_key=None, hvg_flavor='seurat_v3', binning=None, result_binned_key='X_binned', tokenize_name='scBERT', return_pt=True, append_cls=True, include_zero_gene=False, cls_token='<cls>', max_len=16000, pad_token='<pad>', pad_value=-2, cls_appended=True, mask_ratio=0.15, mask_value=-1, preprocessed_loc=None, data_layer_name='X_log1p', label_key='Ground_Truth', batch_label_key=None, cls_nb=5, binarize=None, bins=None, bin_min=None, bin_max=None, save_in_obs=True, auto_map_str_labels=True, map_dict=None, n_splits=1, test_size=None, random_state=2023, shuffle=True, sort_seq_batch=False)


In [19]:
# if this is the first time to run, need this block to init translate=True
# init preprocessor
from scLLM.Dataset.Reader import scReader
data_reader = scReader(dataset_para=dataset_para)
# init vocab
data_reader.init_vocab()

# load data
data_reader.load_adata(loc = data_path,translate=True)

## preprocess
data_reader.preprocess()

scLLM - INFO - Initializing preprocessor ...
scLLM - INFO - use default vocab from dataset_para
scLLM - INFO - load vocab from /home/shi/WorkSpace/projects/scMultiNet_workspace//Experiment/support_data/vocab_16k.json
scLLM - INFO - Load data from anndata object.
scLLM - DEBUG - In original adata with gene 11929
scLLM - DEBUG - In original adata with gene 11929
scLLM - DEBUG - processing 0/16906
scLLM - DEBUG - processing 2000/16906
scLLM - DEBUG - processing 4000/16906
scLLM - DEBUG - processing 6000/16906
scLLM - DEBUG - processing 8000/16906
scLLM - DEBUG - processing 10000/16906
scLLM - DEBUG - processing 12000/16906
scLLM - DEBUG - processing 14000/16906
scLLM - DEBUG - processing 16000/16906
scLLM - INFO - create anndata in scLLM format..
scLLM - DEBUG - restore anndata in scLLM format..
scLLM - INFO - Done.
scLLM - INFO - Preprocessing data with shape: (3754, 16906) ...
scLLM - INFO - Filtering cells by counts ...
scLLM - INFO - Filtered cells: 3754
scLLM - INFO - Normalizing tot

In [20]:
data_reader.adata.n_vars

16906

In [21]:
target_stimulate = "TGFb"
target_cellline = None #"A549" 'DU145', 'MCF7', 'OVCA420'
map_stimulus = False
map_cancer_type = False

print(f"选择前的数据矩阵形状 {data_reader.adata.shape}")
if map_stimulus:
    stimilus_map = data_reader.adata.obs['stimulus']==target_stimulate
    # filter out cells that are not stimulated by TGFb
    new_adata = data_reader.adata[stimilus_map,:]
    data_reader.adata = new_adata

if map_cancer_type:
    cancer_type_map = data_reader.adata.obs['CellLine']==target_cellline
    # filter out cells that are not stimulated by TGFb
    new_adata = data_reader.adata[cancer_type_map,:]
    data_reader.adata = new_adata

print(f"选择后的数据矩阵形状 {data_reader.adata.shape}")

选择前的数据矩阵形状 (3754, 16906)
选择后的数据矩阵形状 (3754, 16906)


In [22]:
# get layer X_log1p
data = data_reader.adata.layers["X_log1p"] 
from scipy.sparse import coo_matrix, csr_matrix
# set nan to 0
if isinstance(data, csr_matrix):
    # 使用 CSR 格式的稀疏矩阵可以直接通过 data.data 访问非零元素
    data.data[np.isnan(data.data)] = 0
    # 注意: 这里没有改变矩阵的稀疏结构，只是将非零元素中的 NaN 值替换为 0
    data.data[np.isinf(data.data)] = data.data.max()
else:
    # 如果 data 不是稀疏矩阵，可以直接使用 numpy 的方法
    data[np.isnan(data)] = 0
    data[np.isinf(data)] = data.max()

if issparse(data):
    # 将稀疏矩阵转换为密集格式，然后计算 NaN 值的数量
    nan_count = np.isnan(data.toarray()).sum()
else:
    # 如果 adata.X 不是稀疏矩阵，直接计算 NaN 值的数量
    nan_count = np.isnan(data).sum()

print(f"NaN values in adata.X: {nan_count}")
data_reader.adata.layers["X_log1p"]  = data 

NaN values in adata.X: 0


In [23]:
# data <0 的个数
if issparse(data):
    # 将稀疏矩阵转换为密集格式，然后计算 NaN 值的数量
    nan_count = np.sum(data.toarray()<0)
else:
    # 如果 adata.X 不是稀疏矩阵，直接计算 NaN 值的数量
    nan_count = np.sum(data<0)

print(f"Negative values in adata.X: {nan_count}")

Negative values in adata.X: 0


In [24]:
label_unique = data_reader.adata.obs[dataset_para.label_key].unique()
print(label_unique)
label_dict = {'fake_lable_0':0,'fake_lable_1':1}

[1 0]


In [25]:
# set fake label
#data_reader.adata.obs[dataset_para.label_key]
revert_dict = {v:k for k,v in label_dict.items()}
# map to str label
data_reader.adata.obs[dataset_para.label_key] = data_reader.adata.obs[dataset_para.label_key].map(revert_dict)

In [26]:
trainset,valset,weights = data_reader.postprocess()

# 输出数据集信息
print("trainset size: ",len(trainset))
print("valset size: ",len(valset)) if valset is not None else print("no valset")
print(label_dict)
print("weights: ",weights)

scLLM - INFO - Map string labels to int automatically.
scLLM - INFO - Mapping from {'fake_lable_1': 0, 'fake_lable_0': 1}
scLLM - INFO - Discritize label Ground_Truth in obs_names


trainset size:  3754
no valset
{'fake_lable_0': 0, 'fake_lable_1': 1}
weights:  None


In [27]:
trainset.sample_id = data_reader.adata.obs.index.to_list()

In [28]:
print(len(trainset.sample_id), len(trainset))

3754 3754


In [29]:
import dill
# 为trainset 添加其他labels

#target_task = f"/TrVal_dataset_GT_{target_stimulate}_{target_cellline}.pkl"
target_task = f"/TrVal_dataset_{target_label}.pkl"
loc = save_root + target_task
# 保存 trainset 到文件，并关联相应labels
with open(loc,"wb") as f:
    dill.dump([trainset,valset,weights,label_dict],f)